In [19]:
#from os import chdir
#from google.colab import drive

#drive.mount("/content/drive", force_remount=True)
#chdir("/content/drive/MyDrive/Eliott/files/")

In [1]:
# IMPORTS

import numpy as np
import pandas as pd




In [2]:
# Ouverture

df_train = pd.read_csv('Corona_NLP_train.csv', sep=',', encoding='latin')

df_test = pd.read_csv('Corona_NLP_test.csv', sep=',', encoding='latin')



In [3]:
df_train.shape

(41157, 6)

In [4]:
df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [5]:
# PREPROCESSING

On vérifie qu'il n'y a pas de null (autre que location) et de ligne dupliquées

In [6]:
df_train.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [7]:
df_train = df_train.drop(['Location'], axis=1)
df_test = df_test.drop(['Location'], axis=1)



In [8]:
df_train['Sentiment'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [9]:
df_train.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

In [10]:
train_df2 = df_train.dropna()
test_df2 = df_test.dropna()



In [11]:
train_df2.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

In [12]:
train_df3 = train_df2.drop_duplicates()
test_df3 = test_df2.drop_duplicates()

In [13]:
train_df3.count()

UserName         41157
ScreenName       41157
TweetAt          41157
OriginalTweet    41157
Sentiment        41157
dtype: int64

On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys


In [14]:
import preprocessor as p


In [15]:
def preprocess_tweet(row):
    text = row['OriginalTweet']
    text = p.clean(text)
    return text

In [16]:
train_df3['OriginalTweet'] = train_df3.apply(preprocess_tweet, axis=1)
test_df3['OriginalTweet'] = test_df3.apply(preprocess_tweet, axis=1)

In [17]:
train_df3.head()


,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment
0,3799,48751,16-03-2020,and and,Neutral
1,3800,48752,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,16-03-2020,"Me, ready to go at supermarket during the outb...",Extremely Negative


In [19]:
train_df3['OriginalTweet'] = train_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
test_df3['OriginalTweet'] = test_df3['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

In [20]:
train_df3.head()


,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment
0,3799,48751,16-03-2020,and and,Neutral
1,3800,48752,16-03-2020,advice talk to your neighbours family to excha...,Positive
2,3801,48753,16-03-2020,coronavirus australia woolworths to give elder...,Positive
3,3802,48754,16-03-2020,my food stock is not the only one which is emp...,Positive
4,3803,48755,16-03-2020,me ready to go at supermarket during the outbr...,Extremely Negative


In [38]:
def give_number_to_class(row):
    sent = row['Sentiment']

    if sent=='Extremely Negative':
        return -1
    elif sent=='Negative':
        return -1
    elif sent=='Neutral':
        return 0
    elif sent=='Positive':
        return 1
    else:
        return 1

In [39]:
train_df3['Sentiment_Number'] = train_df3.apply(give_number_to_class, axis=1)
test_df3['Sentiment_Number'] = test_df3.apply(give_number_to_class, axis=1)

In [40]:
train_df3.head()

,UserName,ScreenName,TweetAt,OriginalTweet,Sentiment,Sentiment_Number
0,3799,48751,16-03-2020,and and,Neutral,0
1,3800,48752,16-03-2020,advice talk to your neighbours family to excha...,Positive,1
2,3801,48753,16-03-2020,coronavirus australia woolworths to give elder...,Positive,1
3,3802,48754,16-03-2020,my food stock is not the only one which is emp...,Positive,1
4,3803,48755,16-03-2020,me ready to go at supermarket during the outbr...,Extremely Negative,-1


In [41]:
np.unique(train_df3["Sentiment"].values)

array(['Extremely Negative', 'Extremely Positive', 'Negative', 'Neutral',
       'Positive'], dtype=object)

In [42]:
X_train = train_df3['OriginalTweet']  
X_test = test_df3['OriginalTweet']  

y_train = train_df3['Sentiment_Number']
y_test = test_df3['Sentiment_Number']

In [43]:
X_train[0]

'and and'

In [44]:
df_train.isnull().sum()

UserName         0
ScreenName       0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

In [45]:
df_train['TweetAt'].values

array(['16-03-2020', '16-03-2020', '16-03-2020', ..., '14-04-2020',
       '14-04-2020', '14-04-2020'], dtype=object)

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(41157, 35525)

In [47]:
X_train_tfidf[1]


<1x35525 sparse matrix of type '<class 'numpy.float64'>'
	with 33 stored elements in Compressed Sparse Row format>

In [48]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import SGDClassifier

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC

grid_params = {
  'sgd_clf__penalty': ['l2', 'l1', 'elasticnet'],
  'sgd_clf__alpha': np.linspace(0.00001, 0.001, 10),
}

pipeline_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('sgd_clf', SGDClassifier()),
])

clf = GridSearchCV(pipeline_clf, grid_params, verbose=1, n_jobs=-1)
clf.fit(X_train, y_train)

print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

# Feed the training data through the pipeline
#text_clf.fit(X_train, y_train) 

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Score:  0.8767646996664226
Best Params:  {'sgd_clf__alpha': 1e-05, 'sgd_clf__penalty': 'l1'}


In [49]:
from sklearn.linear_model import Perceptron

grid_params = {
  'per_clf__penalty': ['l2', 'l1', 'elasticnet'],
  'per_clf__alpha': np.linspace(0.00001, 0.001, 10),
}

pipeline_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('per_clf', Perceptron()),
])

clf = GridSearchCV(pipeline_clf, grid_params, verbose=1, n_jobs=-1)
clf.fit(X_train, y_train)

print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Score:  0.791651326688233
Best Params:  {'per_clf__alpha': 1e-05, 'per_clf__penalty': 'l1'}


In [50]:
from sklearn.svm import LinearSVC

grid_params = {
  'svc_clf__penalty': ['l2', 'l1', 'elasticnet'],
  'svc_clf__loss': ['hinge', 'squared_hinge'],
}

pipeline_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('svc_clf', LinearSVC()),
])

clf = GridSearchCV(pipeline_clf, grid_params, verbose=1, n_jobs=-1)
clf.fit(X_train, y_train)

print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\Honorine\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Honorine\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Honorine\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Honorine\anaconda3\lib\site-packages\sklearn\svm\_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_l

Best Score:  0.8309157326606293
Best Params:  {'svc_clf__loss': 'squared_hinge', 'svc_clf__penalty': 'l2'}


In [ ]:
predictions = text_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

train_acc = round(accuracy_score(predictions, y_test)*100)
train_acc

In [ ]:
len(y_test)

In [ ]:
print(classification_report(predictions, y_test, target_names=train_df3['Sentiment'].unique()))

In [ ]:
X_test[0]

In [ ]:
# ACCURACY
c=0
num=len(predictions)
for k in range(num):
    if(predictions[k]==y_test[k]):
        c+=1

print(f"pourcentage de reussite : {(c*100)/num} ")




In [ ]:
# L2 ERROR

num=len(predictions)
err_tot = 0
for k in range(num):
    #print(predictions[k])
    if(k<20):
        print(f"prediction {predictions[k]}  , realité : {y_test[k]}")
    err_tot += np.square(predictions[k]-y_test[k])
err_tot/=num
err_tot = np.sqrt(err_tot) 
err_tot